In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import timedelta, datetime, time
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from scipy.stats import entropy


In [2]:
path = '/content/drive/MyDrive/Aviator/Data1/bd_pseudo_1.csv'
data = pd.read_csv(path)
print(data)
t = 0
print(len(data))
t0 = int(input("Maximo: "))
tamanho_data = len(data)

odd = []
horario = []
odd_ajustada = []

while t <= t0:
    if t < tamanho_data:
        value = data['odd'][t]
        if isinstance(value, float):
            odd.append(np.nan)  # or any other value to represent missing data
        else:
            aux = value.split('\n\n')
            a1 = aux[0].replace(')I', '')
            a2 = a1.replace(')', '')
            a3 = a2.replace('|', '1')
            a4 = a3.replace('?', '7')
            a5 = re.sub('[^0-9.]', '', a4)
            odd.append(float(a5))
        horario.append(data['hora_criacao'][t])

    t += 1

for name in odd:
    ajuste = float(name - 1.25)
    odd_ajustada.append(ajuste)

horas = []
minutos = []
segundos = []
for item in horario:
    dt = datetime.strptime(item, "%a %b %d %H:%M:%S %Y")
    horas.append(dt.hour)
    minutos.append(dt.minute)
    segundos.append(dt.second)

data1 = pd.DataFrame({'Odd_ajustada': odd, 'hora': horas, 'minutos': minutos, 'segundos': segundos, 'horario': horario})

###Data final
#data = dados_filtrados
data = data1

# Verificar se existem valores NaN na coluna "Odd_ajustada"
has_nan = data['Odd_ajustada'].isna().any()
if has_nan:
    print("Existem valores NaN na coluna 'Odd_ajustada'.")
else:
    print("Não existem valores NaN na coluna 'Odd_ajustada'.")


          id    odd              hora_criacao apostadores
0          0  13.89  Sat May 27 00:28:06 2023        1650
1          1   1.44  Sat May 27 00:28:18 2023       22211
2          2   3.39  Sat May 27 00:28:42 2023        1751
3          3   1.27  Sat May 27 00:28:56 2023        1950
4          4   5.16  Sat May 27 00:29:26 2023        2086
...      ...    ...                       ...         ...
37852  37855   1.53  Fri Oct 20 23:00:27 2023           
37853  37856   1.36  Fri Oct 20 23:00:45 2023        181
37854  37857   1.00  Fri Oct 20 23:00:59 2023       1882
37855  37858   2.17  Fri Oct 20 23:01:20 2023          1
37856  37859   2.12  Fri Oct 20 23:01:46 2023           

[37857 rows x 4 columns]
37857
Maximo: 37857
Não existem valores NaN na coluna 'Odd_ajustada'.


In [3]:
data1 = data

In [4]:
geral_cat = []
for name in data['Odd_ajustada']:
  #print(name)
  if name < 1.05:
    geral_cat.append(1)
  elif name >= 1.05 and name < 1.15:
    geral_cat.append(2)
  elif name >= 1.15 and name < 1.3:
    geral_cat.append(3)
  elif name >= 1.3 and name < 1.45:
    geral_cat.append(4)
  elif name >= 1.45 and name < 1.7:
    geral_cat.append(5)
  elif name >= 1.7 and name < 2.1:
    geral_cat.append(6)
  elif name >= 2.1 and name < 2.6:
    geral_cat.append(7)
  elif name >= 2.6 and name < 3.5:
    geral_cat.append(8)
  elif name >= 3.5 and name < 5:
    geral_cat.append(9)
  elif name >= 5 and name < 10:
    geral_cat.append(10)
  elif name >= 10:
    geral_cat.append(11)
print(len(geral_cat))
geral_cat1 = pd.DataFrame([geral_cat]).T
geral_cat1.columns = ['Odd_Categoria']
data = pd.concat([data, geral_cat1], axis = 1)

data['Odd_Categoria'].describe()

37857


count    37857.000000
mean         6.114536
std          3.090147
min          1.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         11.000000
Name: Odd_Categoria, dtype: float64

In [5]:
# Crie as colunas "entrada" e "saida" deslocando a coluna "horario" para cima
new_data = pd.DataFrame()
new_data['odd_saida'] = data['Odd_Categoria']
new_data['odd_entrada'] = data['Odd_Categoria'].shift(1)

new_data = new_data.drop(new_data.index[0])
# Visualize o DataFrame new_data
print(new_data)

data = pd.concat([data, new_data], axis = 1)

data = data.drop(data.index[0])

       odd_saida  odd_entrada
1              4         11.0
2              8          4.0
3              3          8.0
4             10          3.0
5              8         10.0
...          ...          ...
37852          5         11.0
37853          4          5.0
37854          1          4.0
37855          7          1.0
37856          7          7.0

[37856 rows x 2 columns]


In [6]:
data = new_data.reset_index(drop = True)

In [7]:
data


,odd_saida,odd_entrada
0,4,11.0
1,8,4.0
2,3,8.0
3,10,3.0
4,8,10.0
...,...,...
37851,5,11.0
37852,4,5.0
37853,1,4.0
37854,7,1.0


In [ ]:
t = len(data)
arrayg = []
for i in range(4, t):
  j = i - 4
  array5 = []
  while j <= i:
    array5.append(data['odd_entrada'][j])
    j += 1
  arrayg.append(array5)
arrayg1 = []
for i in range(9, t):
  j = i - 9
  array10 = []
  while j <= i:
    array10.append(data['odd_entrada'][j])
    j += 1
  arrayg1.append(array10)
arrayg2 = []
for i in range(19, t):
  j = i - 19
  array20 = []
  while j <= i:
    array20.append(data['odd_entrada'][j])
    j += 1
  arrayg2.append(array20)
arrayg3 = []
for i in range(39, t):
  j = i - 39
  array40 = []
  while j <= i:
    array40.append(data['odd_entrada'][j])
    j += 1
  arrayg3.append(array40)
arrayg4 = []
for i in range(79, t):
  j = i - 79
  array80 = []
  while j <= i:
    array80.append(data['odd_entrada'][j])
    j += 1
  arrayg4.append(array80)
arrayg5 = []
for i in range(159, t):
  j = i - 159
  array160 = []
  while j <= i:
    array160.append(data['odd_entrada'][j])
    j += 1
  arrayg5.append(array160)
arrayg6 = []
for i in range(319, t):
  j = i - 319
  array320 = []
  while j <= i:
    array320.append(data['odd_entrada'][j])
    j += 1
  arrayg6.append(array320)
arrayg7 = []
for i in range(639, t):
  j = i - 639
  array640 = []
  while j <= i:
    array640.append(data['odd_entrada'][j])
    j += 1
  arrayg7.append(array640)

In [ ]:
print(len(arrayg),len(arrayg1),len(arrayg2),len(arrayg3),len(arrayg4),len(arrayg5),len(arrayg6),len(arrayg7))

37852 37847 37837 37817 37777 37697 37537 37217


In [ ]:
arrayg = arrayg[635:]
arrayg1 = arrayg1[630:]
arrayg2 = arrayg2[620:]
arrayg3 = arrayg3[600:]
arrayg4 = arrayg4[560:]
arrayg5 = arrayg5[480:]
arrayg6 = arrayg6[320:]

In [ ]:
print(len(arrayg),len(arrayg1),len(arrayg2),len(arrayg3),len(arrayg4),len(arrayg5),len(arrayg6),len(arrayg7))

37217 37217 37217 37217 37217 37217 37217 37217


In [ ]:
data = data.iloc[639:].reset_index(drop=True)
print(len(data))

37217


In [ ]:
array = pd.DataFrame({'5entradas': arrayg})
array1 = pd.DataFrame({'10entradas': arrayg1})
array2 = pd.DataFrame({'20entradas': arrayg2})
array3 = pd.DataFrame({'40entradas': arrayg3})
array4 = pd.DataFrame({'80entradas': arrayg4})
array5 = pd.DataFrame({'160entradas': arrayg5})
array6 = pd.DataFrame({'320entradas': arrayg6})
array7 = pd.DataFrame({'640entradas': arrayg7})



In [ ]:
data = pd.concat([data, array, array1, array2, array3, array4, array5, array6, array7], axis = 1)

In [ ]:
data.head()

,odd_saida,odd_entrada,5entradas,10entradas,20entradas,40entradas,80entradas,160entradas,320entradas,640entradas
0,10,10.0,"[6.0, 8.0, 5.0, 9.0, 10.0]","[10.0, 9.0, 1.0, 7.0, 4.0, 6.0, 8.0, 5.0, 9.0,...","[6.0, 5.0, 9.0, 5.0, 11.0, 2.0, 2.0, 6.0, 2.0,...","[11.0, 10.0, 8.0, 4.0, 4.0, 6.0, 11.0, 1.0, 5....","[8.0, 2.0, 4.0, 7.0, 11.0, 9.0, 7.0, 6.0, 9.0,...","[8.0, 2.0, 8.0, 1.0, 2.0, 3.0, 6.0, 10.0, 10.0...","[5.0, 8.0, 8.0, 8.0, 9.0, 11.0, 4.0, 3.0, 3.0,...","[11.0, 4.0, 8.0, 3.0, 10.0, 8.0, 1.0, 6.0, 6.0..."
1,3,10.0,"[8.0, 5.0, 9.0, 10.0, 10.0]","[9.0, 1.0, 7.0, 4.0, 6.0, 8.0, 5.0, 9.0, 10.0,...","[5.0, 9.0, 5.0, 11.0, 2.0, 2.0, 6.0, 2.0, 5.0,...","[10.0, 8.0, 4.0, 4.0, 6.0, 11.0, 1.0, 5.0, 11....","[2.0, 4.0, 7.0, 11.0, 9.0, 7.0, 6.0, 9.0, 7.0,...","[2.0, 8.0, 1.0, 2.0, 3.0, 6.0, 10.0, 10.0, 3.0...","[8.0, 8.0, 8.0, 9.0, 11.0, 4.0, 3.0, 3.0, 9.0,...","[4.0, 8.0, 3.0, 10.0, 8.0, 1.0, 6.0, 6.0, 5.0,..."
2,5,3.0,"[5.0, 9.0, 10.0, 10.0, 3.0]","[1.0, 7.0, 4.0, 6.0, 8.0, 5.0, 9.0, 10.0, 10.0...","[9.0, 5.0, 11.0, 2.0, 2.0, 6.0, 2.0, 5.0, 10.0...","[8.0, 4.0, 4.0, 6.0, 11.0, 1.0, 5.0, 11.0, 3.0...","[4.0, 7.0, 11.0, 9.0, 7.0, 6.0, 9.0, 7.0, 11.0...","[8.0, 1.0, 2.0, 3.0, 6.0, 10.0, 10.0, 3.0, 6.0...","[8.0, 8.0, 9.0, 11.0, 4.0, 3.0, 3.0, 9.0, 3.0,...","[8.0, 3.0, 10.0, 8.0, 1.0, 6.0, 6.0, 5.0, 8.0,..."
3,8,5.0,"[9.0, 10.0, 10.0, 3.0, 5.0]","[7.0, 4.0, 6.0, 8.0, 5.0, 9.0, 10.0, 10.0, 3.0...","[5.0, 11.0, 2.0, 2.0, 6.0, 2.0, 5.0, 10.0, 9.0...","[4.0, 4.0, 6.0, 11.0, 1.0, 5.0, 11.0, 3.0, 9.0...","[7.0, 11.0, 9.0, 7.0, 6.0, 9.0, 7.0, 11.0, 2.0...","[1.0, 2.0, 3.0, 6.0, 10.0, 10.0, 3.0, 6.0, 5.0...","[8.0, 9.0, 11.0, 4.0, 3.0, 3.0, 9.0, 3.0, 5.0,...","[3.0, 10.0, 8.0, 1.0, 6.0, 6.0, 5.0, 8.0, 9.0,..."
4,2,8.0,"[10.0, 10.0, 3.0, 5.0, 8.0]","[4.0, 6.0, 8.0, 5.0, 9.0, 10.0, 10.0, 3.0, 5.0...","[11.0, 2.0, 2.0, 6.0, 2.0, 5.0, 10.0, 9.0, 1.0...","[4.0, 6.0, 11.0, 1.0, 5.0, 11.0, 3.0, 9.0, 2.0...","[11.0, 9.0, 7.0, 6.0, 9.0, 7.0, 11.0, 2.0, 4.0...","[2.0, 3.0, 6.0, 10.0, 10.0, 3.0, 6.0, 5.0, 7.0...","[9.0, 11.0, 4.0, 3.0, 3.0, 9.0, 3.0, 5.0, 4.0,...","[10.0, 8.0, 1.0, 6.0, 6.0, 5.0, 8.0, 9.0, 10.0..."


import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_0, *['odd_saida'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['odd_entrada'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plot(df, x_colname, y_colname, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(6 * figscale, 6 * figscale))
  df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha)
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plot(_df_2, *['odd_saida', 'odd_entrada'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_3, *['odd_saida'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_4, *['odd_entrada'], **{})
chart

In [ ]:
data.columns

Index(['odd_saida', 'odd_entrada', '5entradas', '10entradas', '20entradas',
       '40entradas', '80entradas', '160entradas', '320entradas',
       '640entradas'],
      dtype='object')

In [ ]:
data2 = data

In [ ]:
from scipy.stats import chi2_contingency
# Converta as listas da coluna 'entrada5' em strings
data['5entradas'] = data['5entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed = pd.crosstab(data['odd_saida'], data['5entradas'])
chi2, p, _, _ = chi2_contingency(observed)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e 'entrada5'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e 'entrada5'.")

Chi-squared: 331211.12663212925
P-value: 0.5577343759923088
Não há uma associação significativa entre 'odd_saida' e 'entrada5'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['10entradas'] = data['10entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed1 = pd.crosstab(data['odd_saida'], data['10entradas'])
chi2, p, _, _ = chi2_contingency(observed1)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e 'entrada5'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e 'entrada5'.")

Chi-squared: 372170.0000000002
P-value: 0.49506776027286753
Não há uma associação significativa entre 'odd_saida' e 'entrada5'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['20entradas'] = data['20entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed2 = pd.crosstab(data['odd_saida'], data['20entradas'])
chi2, p, _, _ = chi2_contingency(observed2)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e 'entrada5'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e 'entrada5'.")

Chi-squared: 372169.99999999994
P-value: 0.4950677602729752
Não há uma associação significativa entre 'odd_saida' e 'entrada5'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['40entradas'] = data['40entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed3 = pd.crosstab(data['odd_saida'], data['40entradas'])
chi2, p, _, _ = chi2_contingency(observed3)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e '40entradas'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e '40entradas'.")

Chi-squared: 372169.99999999994
P-value: 0.4950677602729752
Não há uma associação significativa entre 'odd_saida' e '40entradas'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['80entradas'] = data['80entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed4 = pd.crosstab(data['odd_saida'], data['80entradas'])
chi2, p, _, _ = chi2_contingency(observed4)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e '80entradas'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e '80entradas'.")

Chi-squared: 372170.0
P-value: 0.4950677602729483
Não há uma associação significativa entre 'odd_saida' e '80entradas'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['160entradas'] = data['160entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed5 = pd.crosstab(data['odd_saida'], data['160entradas'])
chi2, p, _, _ = chi2_contingency(observed5)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e '160entradas'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e '160entradas'.")

Chi-squared: 372170.0
P-value: 0.4950677602729483
Não há uma associação significativa entre 'odd_saida' e '160entradas'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['320entradas'] = data['320entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed6 = pd.crosstab(data['odd_saida'], data['320entradas'])
chi2, p, _, _ = chi2_contingency(observed6)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e '320entradas'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e '320entradas'.")

Chi-squared: 372170.0
P-value: 0.4950677602729483
Não há uma associação significativa entre 'odd_saida' e '320entradas'.


In [ ]:
# Converta as listas da coluna 'entrada5' em strings
data['640entradas'] = data['640entradas'].apply(lambda x: str(x))

# Agora, vamos calcular o teste de qui-quadrado
observed7 = pd.crosstab(data['odd_saida'], data['640entradas'])
chi2, p, _, _ = chi2_contingency(observed7)

print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")

alpha = 0.05  # Nível de significância
if p < alpha:
    print("Há uma associação significativa entre 'odd_saida' e '640entradas'.")
else:
    print("Não há uma associação significativa entre 'odd_saida' e '640entradas'.")

Chi-squared: 372169.99999999994
P-value: 0.4950677602729752
Não há uma associação significativa entre 'odd_saida' e '640entradas'.
